In [4]:
# pip install upgini & catboost
from os.path import exists
import pandas as pd

In [5]:
# Import data
df_path = "train.csv.zip" if exists("train.csv.zip") else "https://github.com/upgini/upgini/raw/main/notebooks/train.csv.zip"
df = pd.read_csv(df_path)
df = df.sample(n=19000, random_state=0)  # random sample of 19k datapts

In [7]:
# Data processing
# Change store & item as strings
df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)

# Convert date into pandas datetime
df["date"] = pd.to_datetime(df["date"])

df.sort_values("date", inplace=True)
df.reset_index(inplace=True, drop=True)
df.head()

,date,store,item,sales
0,2013-01-01,7,5,5
1,2013-01-01,4,9,19
2,2013-01-01,1,33,37
3,2013-01-01,3,41,14
4,2013-01-01,5,24,26


In [ ]:
# Split train and test data
# Train: 2013-2016
# Test: 2017
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]